In [ ]:



import requests
import json
import time

base_url = 'https://content.guardianapis.com/search'
all_articles = []
page_size = 50
current_page = 1
total_pages = 1
api_key = 'addyours'

while current_page <= total_pages:
    params = {
        "q": "Narendra Modi",
        "api-key": api_key,
        "page-size": page_size,
        "page": current_page,
        "show-fields": "all",
        "show-tags": "all",
        "order-by": "newest"
    }

    try:
        response = requests.get(base_url, params=params)
        data = response.json()

        # Pagination and extraction
        total_pages = data.get("response", {}).get("pages", 1)
        results = data.get("response", {}).get("results", [])
        total_results = data.get("response", {}).get("total", 0)

        all_articles.extend(results)

        print(f"Fetched page {current_page}/{total_pages} — Total articles fetched: {len(all_articles)}/{total_results}")

        if current_page < total_pages:
            current_page += 1
            time.sleep(0.5)  # Be nice to the API
        else:
            break

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        break
    except KeyError as e:
        print(f"Error parsing JSON response, missing key: {e}")
        print(f"Problematic response: {data}")
        break

# Optionally save the result
with open("narendra_modi_guardian_articles.json", "w", encoding="utf-8") as f:
    json.dump(all_articles, f, indent=2)

print("✅ All articles saved successfully.")


Fetched page 1/81 — Total articles fetched: 50/4039
Fetched page 2/81 — Total articles fetched: 100/4039
Fetched page 3/81 — Total articles fetched: 150/4039
Fetched page 4/81 — Total articles fetched: 200/4039


KeyboardInterrupt: 

In [17]:
import pandas as pd

# Convert articles list to a DataFrame
df = pd.json_normalize(all_articles)
df.head()


,id,type,sectionId,sectionName,webPublicationDate,webTitle,webUrl,apiUrl,tags,isHosted,...,fields.showAffiliateLinks,fields.bylineHtml,fields.showTableOfContents,fields.newspaperPageNumber,fields.newspaperEditionDate,fields.commentCloseDate,fields.commentable,fields.sensitive,fields.starRating,fields.displayHint
0,world/live/2025/jul/29/aidrops-aid-futile-msf-...,liveblog,world,World news,2025-07-29T10:04:25Z,"Famine now unfolding in Gaza, says UN-backed m...",https://www.theguardian.com/world/live/2025/ju...,https://content.guardianapis.com/world/live/20...,"[{'id': 'world/series/middle-east-crisis', 'ty...",False,...,false,"<a href=""profile/hamish-mackay"">Hamish Mackay<...",false,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,music/2025/jul/28/theyre-rowdy-theyre-vibing-i...,article,music,Music,2025-07-28T04:00:42Z,‘They’re rowdy. They’re vibing. I rip my shirt...,https://www.theguardian.com/music/2025/jul/28/...,https://content.guardianapis.com/music/2025/ju...,"[{'id': 'music/rap', 'type': 'keyword', 'secti...",False,...,true,"<a href=""profile/ciaran-thapar"">Ciaran Thapar</a>",false,8,2025-07-28T00:00:00Z,NaN,NaN,NaN,NaN,NaN
2,politics/live/2025/jul/24/keir-starmer-narendr...,liveblog,politics,Politics,2025-07-24T14:55:06Z,Jeremy Corbyn confirms he is setting up new po...,https://www.theguardian.com/politics/live/2025...,https://content.guardianapis.com/politics/live...,[{'id': 'politics/series/politics-live-with-an...,False,...,false,"<a href=""profile/tom-ambrose"">Tom Ambrose</a> ...",false,NaN,NaN,2025-07-24T14:15:00Z,true,NaN,NaN,NaN
3,politics/2025/jul/24/jeremy-corbyn-and-zarah-s...,article,politics,Politics,2025-07-24T12:27:52Z,Jeremy Corbyn and Zarah Sultana agree to launc...,https://www.theguardian.com/politics/2025/jul/...,https://content.guardianapis.com/politics/2025...,"[{'id': 'politics/politics', 'type': 'keyword'...",False,...,false,"<a href=""profile/peterwalker"">Peter Walker</a>...",false,1,2025-07-25T00:00:00Z,NaN,NaN,NaN,NaN,NaN
4,politics/2025/jul/24/starmer-modi-uk-india-fre...,article,politics,Politics,2025-07-24T12:01:09Z,Starmer and Modi hail ‘historic day’ as UK-Ind...,https://www.theguardian.com/politics/2025/jul/...,https://content.guardianapis.com/politics/2025...,"[{'id': 'politics/trade-policy', 'type': 'keyw...",False,...,false,"<a href=""profile/eleni-courea"">Eleni Courea</a...",false,4,2025-07-25T00:00:00Z,NaN,NaN,NaN,NaN,NaN


In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from urllib.parse import quote_plus

# Connection details
db_user = 'postgres'
db_password =   # Password URL-encoded
db_host = 'localhost'
db_port = '5432'
db_name = 'postgres'  # Or 'postgres', if you're using default DB

# Create engine
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')


In [19]:

import pandas as pd
import json

def serialize_complex_types(val):
    if isinstance(val, (dict, list)):
        return json.dumps(val)
    return val

df = df.applymap(serialize_complex_types)
df.to_sql("guardian_articles", engine, if_exists='replace', index=False)


C:\Users\ASUS\AppData\Local\Temp\ipykernel_1028\1912795455.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(serialize_complex_types)


200

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

engine = create_engine('postgresql://postgres:@localhost:5432/postgres')
df = pd.read_sql('SELECT * FROM guardian_articles', engine)

# Export to CSV
df.to_csv('guardian_articles.csv', index=False)
